# Advanced Analysis - Sensitivity and Virtual Experiments

This notebook demonstrates advanced analysis capabilities for optimizing 3D printing processes and understanding parameter sensitivity. Learn how to:

- **Perform sensitivity analysis** (local, Morris screening, Sobol indices)
- **Design virtual experiments** (Full factorial, LHS, CCD, Box-Behnken)
- **Optimize process parameters** for desired structure properties
- **Model response surfaces** to predict structure from process parameters
- **Multi-objective optimization** for balanced performance

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Identify which parameters most affect analysis results
2. Design efficient experiments using statistical methods
3. Optimize process parameters for target structure properties
4. Understand process-structure-performance relationships
5. Perform multi-objective optimization

## ⚠️ Prerequisites

- **Notebook 01**: Basic understanding of loading and segmenting volumes
- **Notebook 02**: Understanding of preprocessing and filtering
- **Notebook 03**: Understanding of morphological analysis
- **Notebook 04**: Understanding of experimental analysis
- **Required packages**: Same as previous notebooks
- **Segmented volume**: Binary segmented volume ready for analysis

## 📖 Usage

1. Run all cells to initialize the widgets
2. Load a segmented volume
3. Configure sensitivity analysis parameters
4. Run sensitivity analysis to identify key parameters
5. Design virtual experiments using DoE methods
6. Optimize process parameters for target metrics
7. Explore response surfaces and optimization results


## 1. Setup and Imports


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
from typing import Dict, List, Optional, Tuple, Any

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Advanced Analysis - Sensitivity and Virtual Experiments")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")


📦 Advanced Analysis - Sensitivity and Virtual Experiments
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True


## 2. Load Framework Modules


In [ ]:
# Load analysis modules
try:
    from src.analyzer import XCTAnalyzer
    from src.analysis.sensitivity_analysis import (
        parameter_sweep, local_sensitivity,
        morris_screening, sobol_indices
    )
    from src.analysis.virtual_experiments import (
        full_factorial_design, latin_hypercube_sampling,
        central_composite_design, box_behnken_design,
        run_virtual_experiment, optimize_process_parameters
    )
    from src.core.metrics import compute_all_metrics
    from src.utils.utils import load_volume, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Advanced Analysis Dashboard

Use the interactive widgets below to perform sensitivity analysis and virtual experiments.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    analyzer = None
    current_volume = None
    sensitivity_results = {}
    virtual_experiment_results = {}
    
    # ============================================
    # Section 1: Data Loading
    # ============================================
    
    file_path_text = widgets.Text(
        value='',
        placeholder='Enter file path to segmented volume',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    file_format_dropdown = widgets.Dropdown(
        options=['Auto-detect', 'DICOM', 'TIFF', 'RAW', 'NIfTI', 'NumPy'],
        value='Auto-detect',
        description='Format:',
        style={'description_width': 'initial'}
    )
    
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    
    load_button = widgets.Button(
        description='📂 Load Volume',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    volume_info_display = widgets.HTML(
        value="<p><i>No volume loaded</i></p>",
        layout=widgets.Layout(height='100px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Sensitivity Analysis Parameters
    # ============================================
    
    sensitivity_method = widgets.Dropdown(
        options=['Parameter Sweep', 'Local Sensitivity', 'Morris Screening', 'Sobol Indices'],
        value='Parameter Sweep',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    metric_to_analyze = widgets.Dropdown(
        options=['void_fraction', 'relative_density', 'surface_area', 'volume'],
        value='void_fraction',
        description='Metric:',
        style={'description_width': 'initial'}
    )
    
    # Parameter ranges for sensitivity
    param1_name = widgets.Text(value='threshold', description='Param 1:', style={'description_width': 'initial'})
    param1_min = widgets.FloatText(value=0.3, description='Min:', style={'description_width': 'initial'})
    param1_max = widgets.FloatText(value=0.5, description='Max:', style={'description_width': 'initial'})
    param1_steps = widgets.IntText(value=5, description='Steps:', style={'description_width': 'initial'})
    
    param2_name = widgets.Text(value='', description='Param 2:', style={'description_width': 'initial'}, disabled=True)
    param2_min = widgets.FloatText(value=0.0, description='Min:', style={'description_width': 'initial'}, disabled=True)
    param2_max = widgets.FloatText(value=1.0, description='Max:', style={'description_width': 'initial'}, disabled=True)
    param2_steps = widgets.IntText(value=5, description='Steps:', style={'description_width': 'initial'}, disabled=True)
    
    n_trajectories = widgets.IntText(
        value=10,
        description='Trajectories (Morris):',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    n_samples_sobol = widgets.IntText(
        value=1000,
        description='Samples (Sobol):',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    run_sensitivity_button = widgets.Button(
        description='🔬 Run Sensitivity Analysis',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    sensitivity_results_display = widgets.HTML(
        value="<p><i>No sensitivity analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    sensitivity_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 3: Virtual Experiments (DoE)
    # ============================================
    
    doe_method = widgets.Dropdown(
        options=['Full Factorial', 'Latin Hypercube', 'Central Composite', 'Box-Behnken'],
        value='Full Factorial',
        description='DoE Method:',
        style={'description_width': 'initial'}
    )
    
    # Process parameters for virtual experiments
    process_param1_name = widgets.Text(value='extrusion_temp', description='Param 1:', style={'description_width': 'initial'})
    process_param1_min = widgets.FloatText(value=200.0, description='Min:', style={'description_width': 'initial'})
    process_param1_max = widgets.FloatText(value=240.0, description='Max:', style={'description_width': 'initial'})
    process_param1_levels = widgets.Text(value='200,220,240', description='Levels:', style={'description_width': 'initial'})
    
    process_param2_name = widgets.Text(value='print_speed', description='Param 2:', style={'description_width': 'initial'})
    process_param2_min = widgets.FloatText(value=20.0, description='Min:', style={'description_width': 'initial'})
    process_param2_max = widgets.FloatText(value=40.0, description='Max:', style={'description_width': 'initial'})
    process_param2_levels = widgets.Text(value='20,30,40', description='Levels:', style={'description_width': 'initial'})
    
    process_param3_name = widgets.Text(value='', description='Param 3:', style={'description_width': 'initial'}, disabled=True)
    process_param3_min = widgets.FloatText(value=0.0, description='Min:', style={'description_width': 'initial'}, disabled=True)
    process_param3_max = widgets.FloatText(value=1.0, description='Max:', style={'description_width': 'initial'}, disabled=True)
    process_param3_levels = widgets.Text(value='', description='Levels:', style={'description_width': 'initial'}, disabled=True)
    
    n_samples_lhs = widgets.IntText(
        value=50,
        description='Samples (LHS):',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    generate_doe_button = widgets.Button(
        description='📊 Generate DoE',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    doe_results_display = widgets.HTML(
        value="<p><i>No DoE generated</i></p>",
        layout=widgets.Layout(height='150px', overflow='auto')
    )
    
    doe_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 4: Optimization
    # ============================================
    
    optimization_objective = widgets.Dropdown(
        options=['Maximize void_fraction', 'Minimize void_fraction', 'Maximize relative_density', 'Custom'],
        value='Maximize void_fraction',
        description='Objective:',
        style={'description_width': 'initial'}
    )
    
    optimization_method = widgets.Dropdown(
        options=['Differential Evolution', 'Nelder-Mead', 'BFGS'],
        value='Differential Evolution',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    run_optimization_button = widgets.Button(
        description='🎯 Optimize Parameters',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )
    
    optimization_results_display = widgets.HTML(
        value="<p><i>No optimization results</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    optimization_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


In [4]:
if WIDGETS_AVAILABLE:
    
    import logging
    logger = logging.getLogger(__name__)
    
    def load_volume_callback(button):
        """Load segmented volume"""
        global analyzer, current_volume
        
        file_path = file_path_text.value.strip()
        if not file_path:
            status_display.value = "<p style='color: red;'>Please enter a file path</p>"
            return
        
        file_path_obj = Path(file_path)
        if not file_path_obj.exists():
            data_path = project_root / 'data' / file_path
            if data_path.exists():
                file_path_obj = data_path
            else:
                status_display.value = f"<p style='color: red;'>File not found: {file_path}</p>"
                return
        
        status_display.value = "<p>Loading volume...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            analyzer = XCTAnalyzer(voxel_size=voxel_size, target_unit='mm')
            progress_bar.value = 40
            
            analyzer.load_volume(str(file_path_obj), normalize=True)
            current_volume = analyzer.volume
            progress_bar.value = 80
            
            info_html = f"""
            <h4>Volume Information</h4>
            <p><b>Shape:</b> {analyzer.volume.shape}</p>
            <p><b>Voxel Size:</b> {voxel_size} mm</p>
            <p><b>Volume Size:</b> {analyzer.volume.nbytes / (1024**2):.2f} MB</p>
            """
            volume_info_display.value = info_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Volume loaded successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error loading volume: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def create_analysis_function(metric_name):
        """Create analysis function for sensitivity analysis"""
        def analysis_func(params):
            """Analysis function that computes metrics"""
            try:
                if current_volume is None:
                    return {metric_name: np.nan}
                
                voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
                
                # For demonstration, we'll use a simplified analysis
                # In practice, this would apply params to segmentation/preprocessing
                metrics = compute_all_metrics(current_volume, voxel_size)
                return metrics
            except Exception as e:
                logger.warning(f"Analysis failed: {e}")
                return {metric_name: np.nan}
        return analysis_func
    
    def run_sensitivity_callback(button):
        """Run sensitivity analysis"""
        global sensitivity_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Running sensitivity analysis...</p>"
        progress_bar.value = 10
        
        try:
            method = sensitivity_method.value
            metric_name = metric_to_analyze.value
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            
            # Create analysis function
            analysis_func = create_analysis_function(metric_name)
            
            # Get base metrics
            base_metrics = compute_all_metrics(current_volume, voxel_size)
            base_value = base_metrics.get(metric_name, 0.0)
            
            if method == 'Parameter Sweep':
                progress_bar.value = 20
                
                # Prepare parameter ranges
                param_ranges = {}
                base_params = {}
                
                # Param 1
                if param1_name.value:
                    param1_range = np.linspace(float(param1_min.value), float(param1_max.value), int(param1_steps.value))
                    param_ranges[param1_name.value] = param1_range
                    base_params[param1_name.value] = np.mean(param1_range)
                
                # Param 2 (if enabled)
                if param2_name.value and not param2_name.disabled:
                    param2_range = np.linspace(float(param2_min.value), float(param2_max.value), int(param2_steps.value))
                    param_ranges[param2_name.value] = param2_range
                    base_params[param2_name.value] = np.mean(param2_range)
                
                if not param_ranges:
                    status_display.value = "<p style='color: red;'>Please specify at least one parameter</p>"
                    return
                
                # Run parameter sweep
                sweep_result = parameter_sweep(base_params, param_ranges, analysis_func, metric_name)
                progress_bar.value = 80
                
                sensitivity_results = sweep_result
                
                # Display results
                df = sweep_result['results']
                html = f"""
                <h4>🔬 Parameter Sweep Results</h4>
                <p><b>Metric:</b> {metric_name}</p>
                <p><b>Base Value:</b> {base_value:.4f}</p>
                <p><b>Number of Runs:</b> {sweep_result['n_runs']}</p>
                <p><b>Parameter Ranges:</b> {list(param_ranges.keys())}</p>
                <h5>Sensitivity Rankings:</h5>
                <ul>
                """
                for param_name, sens_data in sweep_result['sensitivity'].items():
                    html += f"<li><b>{param_name}:</b> Correlation = {sens_data['correlation']:.3f}, Range Effect = {sens_data['range_effect']:.4f}</li>"
                html += "</ul>"
                sensitivity_results_display.value = html
                
                # Visualize
                with sensitivity_visualization:
                    clear_output()
                    n_params = len(sweep_result['param_names'])
                    if n_params == 1:
                        fig, axes = plt.subplots(1, 1, figsize=(10, 6))
                        param_name = sweep_result['param_names'][0]
                        axes.plot(df[param_name], df[metric_name], 'o-', linewidth=2, markersize=8)
                        axes.axhline(base_value, color='red', linestyle='--', label='Base Value')
                        axes.set_xlabel(param_name, fontsize=11)
                        axes.set_ylabel(metric_name, fontsize=11)
                        axes.set_title(f'Sensitivity: {metric_name} vs {param_name}', fontsize=12, fontweight='bold')
                        axes.legend()
                        axes.grid(True, alpha=0.3)
                    elif n_params == 2:
                        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                        for idx, param_name in enumerate(sweep_result['param_names']):
                            axes[idx].plot(df[param_name], df[metric_name], 'o-', linewidth=2, markersize=8)
                            axes[idx].axhline(base_value, color='red', linestyle='--', label='Base Value')
                            axes[idx].set_xlabel(param_name, fontsize=11)
                            axes[idx].set_ylabel(metric_name, fontsize=11)
                            axes[idx].set_title(f'{metric_name} vs {param_name}', fontsize=12, fontweight='bold')
                            axes[idx].legend()
                            axes[idx].grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.show()
                
            elif method == 'Local Sensitivity':
                progress_bar.value = 20
                
                if not param1_name.value:
                    status_display.value = "<p style='color: red;'>Please specify a parameter</p>"
                    return
                
                # Run local sensitivity
                base_params = {param1_name.value: (float(param1_min.value) + float(param1_max.value)) / 2.0}
                local_result = local_sensitivity(base_params, param1_name.value, analysis_func, metric_name)
                progress_bar.value = 80
                
                sensitivity_results = {'local': local_result}
                
                # Display results
                html = f"""
                <h4>🔬 Local Sensitivity Results</h4>
                <p><b>Parameter:</b> {local_result['parameter']}</p>
                <p><b>Base Value:</b> {local_result['base_value']:.4f}</p>
                <p><b>Base Metric:</b> {local_result['base_metric']:.4f}</p>
                <p><b>Derivative:</b> {local_result['derivative']:.6f}</p>
                <p><b>Relative Sensitivity:</b> {local_result['relative_sensitivity']:.4f}</p>
                """
                sensitivity_results_display.value = html
                
                # Visualize
                with sensitivity_visualization:
                    clear_output()
                    fig, axes = plt.subplots(1, 1, figsize=(10, 6))
                    param_vals = [local_result['base_value'] * 0.9, local_result['base_value'], local_result['base_value'] * 1.1]
                    metric_vals = [
                        local_result['backward_metric'],
                        local_result['base_metric'],
                        local_result['forward_metric']
                    ]
                    axes.plot(param_vals, metric_vals, 'o-', linewidth=2, markersize=8)
                    axes.axvline(local_result['base_value'], color='red', linestyle='--', alpha=0.5)
                    axes.set_xlabel(local_result['parameter'], fontsize=11)
                    axes.set_ylabel(metric_name, fontsize=11)
                    axes.set_title(f'Local Sensitivity: {metric_name} vs {local_result["parameter"]}', fontsize=12, fontweight='bold')
                    axes.grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.show()
                
            elif method == 'Morris Screening':
                progress_bar.value = 20
                
                # Prepare parameter bounds
                param_bounds = {}
                if param1_name.value:
                    param_bounds[param1_name.value] = (float(param1_min.value), float(param1_max.value))
                if param2_name.value and not param2_name.disabled:
                    param_bounds[param2_name.value] = (float(param2_min.value), float(param2_max.value))
                
                if not param_bounds:
                    status_display.value = "<p style='color: red;'>Please specify at least one parameter</p>"
                    return
                
                # Run Morris screening
                morris_result = morris_screening(
                    param_bounds, analysis_func,
                    n_trajectories=int(n_trajectories.value),
                    metric_name=metric_name
                )
                progress_bar.value = 80
                
                sensitivity_results = morris_result
                
                # Display results
                html = f"""
                <h4>🔬 Morris Screening Results</h4>
                <p><b>Metric:</b> {metric_name}</p>
                <p><b>Number of Trajectories:</b> {n_trajectories.value}</p>
                <h5>Morris Indices (μ*, σ):</h5>
                <ul>
                """
                for param_name, indices in morris_result['indices'].items():
                    html += f"<li><b>{param_name}:</b> μ* = {indices['mu_star']:.4f}, σ = {indices['sigma']:.4f}</li>"
                html += "</ul>"
                sensitivity_results_display.value = html
                
                # Visualize
                with sensitivity_visualization:
                    clear_output()
                    fig, axes = plt.subplots(1, 1, figsize=(10, 6))
                    param_names = list(morris_result['indices'].keys())
                    mu_stars = [morris_result['indices'][p]['mu_star'] for p in param_names]
                    sigmas = [morris_result['indices'][p]['sigma'] for p in param_names]
                    
                    axes.scatter(mu_stars, sigmas, s=200, alpha=0.7)
                    for i, param_name in enumerate(param_names):
                        axes.annotate(param_name, (mu_stars[i], sigmas[i]), xytext=(5, 5), textcoords='offset points')
                    axes.set_xlabel('μ* (Mean Absolute Effect)', fontsize=11)
                    axes.set_ylabel('σ (Standard Deviation)', fontsize=11)
                    axes.set_title('Morris Screening: Parameter Importance', fontsize=12, fontweight='bold')
                    axes.grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Sensitivity analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in sensitivity analysis: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    print("✅ Callback functions created!")


✅ Callback functions created!


In [5]:
if WIDGETS_AVAILABLE:
    
    def generate_doe_callback(button):
        """Generate Design of Experiments"""
        global virtual_experiment_results
        
        status_display.value = "<p>Generating DoE...</p>"
        progress_bar.value = 20
        
        try:
            method = doe_method.value
            
            if method == 'Full Factorial':
                # Prepare factors
                factors = {}
                if process_param1_name.value:
                    levels1 = [float(x.strip()) for x in process_param1_levels.value.split(',')]
                    factors[process_param1_name.value] = levels1
                if process_param2_name.value:
                    levels2 = [float(x.strip()) for x in process_param2_levels.value.split(',')]
                    factors[process_param2_name.value] = levels2
                if process_param3_name.value and not process_param3_name.disabled:
                    levels3 = [float(x.strip()) for x in process_param3_levels.value.split(',')]
                    factors[process_param3_name.value] = levels3
                
                if not factors:
                    status_display.value = "<p style='color: red;'>Please specify at least one parameter</p>"
                    return
                
                design = full_factorial_design(factors)
                progress_bar.value = 80
                
            elif method == 'Latin Hypercube':
                param_bounds = {}
                if process_param1_name.value:
                    param_bounds[process_param1_name.value] = (float(process_param1_min.value), float(process_param1_max.value))
                if process_param2_name.value:
                    param_bounds[process_param2_name.value] = (float(process_param2_min.value), float(process_param2_max.value))
                if process_param3_name.value and not process_param3_name.disabled:
                    param_bounds[process_param3_name.value] = (float(process_param3_min.value), float(process_param3_max.value))
                
                if not param_bounds:
                    status_display.value = "<p style='color: red;'>Please specify at least one parameter</p>"
                    return
                
                design = latin_hypercube_sampling(param_bounds, n_samples=int(n_samples_lhs.value))
                progress_bar.value = 80
                
            elif method == 'Central Composite':
                factors = {}
                if process_param1_name.value:
                    factors[process_param1_name.value] = (float(process_param1_min.value), float(process_param1_max.value))
                if process_param2_name.value:
                    factors[process_param2_name.value] = (float(process_param2_min.value), float(process_param2_max.value))
                if process_param3_name.value and not process_param3_name.disabled:
                    factors[process_param3_name.value] = (float(process_param3_min.value), float(process_param3_max.value))
                
                if not factors:
                    status_display.value = "<p style='color: red;'>Please specify at least one parameter</p>"
                    return
                
                design = central_composite_design(factors)
                progress_bar.value = 80
                
            elif method == 'Box-Behnken':
                factors = {}
                if process_param1_name.value:
                    factors[process_param1_name.value] = (float(process_param1_min.value), float(process_param1_max.value))
                if process_param2_name.value:
                    factors[process_param2_name.value] = (float(process_param2_min.value), float(process_param2_max.value))
                if process_param3_name.value and not process_param3_name.disabled:
                    factors[process_param3_name.value] = (float(process_param3_min.value), float(process_param3_max.value))
                
                if not factors or len(factors) < 3:
                    status_display.value = "<p style='color: red;'>Box-Behnken requires at least 3 parameters</p>"
                    return
                
                design = box_behnken_design(factors)
                progress_bar.value = 80
            
            virtual_experiment_results['design'] = design
            
            # Display results
            html = f"""
            <h4>📊 DoE Design Generated</h4>
            <p><b>Method:</b> {method}</p>
            <p><b>Number of Experiments:</b> {len(design)}</p>
            <p><b>Parameters:</b> {list(design.columns)}</p>
            <h5>First 5 Experiments:</h5>
            {design.head().to_html(classes='table table-striped', table_id='doe_table')}
            """
            doe_results_display.value = html
            
            # Visualize
            with doe_visualization:
                clear_output()
                n_params = len(design.columns)
                if n_params >= 2:
                    fig, axes = plt.subplots(1, min(2, n_params-1), figsize=(14, 6))
                    if n_params == 2:
                        axes = [axes]
                    param_names = list(design.columns)
                    for idx in range(min(2, n_params-1)):
                        axes[idx].scatter(design[param_names[idx]], design[param_names[idx+1]], 
                                        s=100, alpha=0.6, edgecolors='black')
                        axes[idx].set_xlabel(param_names[idx], fontsize=11)
                        axes[idx].set_ylabel(param_names[idx+1], fontsize=11)
                        axes[idx].set_title(f'{method} Design: {param_names[idx]} vs {param_names[idx+1]}', 
                                          fontsize=12, fontweight='bold')
                        axes[idx].grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ DoE generated successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error generating DoE: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def run_optimization_callback(button):
        """Run parameter optimization"""
        global virtual_experiment_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Running optimization...</p>"
        progress_bar.value = 10
        
        try:
            objective = optimization_objective.value
            method = optimization_method.value
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            
            # Determine objective function
            if 'Maximize void_fraction' in objective:
                target_metric = 'void_fraction'
                maximize = True
            elif 'Minimize void_fraction' in objective:
                target_metric = 'void_fraction'
                maximize = False
            elif 'Maximize relative_density' in objective:
                target_metric = 'relative_density'
                maximize = True
            else:
                target_metric = 'void_fraction'
                maximize = True
            
            # Prepare parameter bounds
            param_bounds = []
            param_names = []
            if process_param1_name.value:
                param_bounds.append((float(process_param1_min.value), float(process_param1_max.value)))
                param_names.append(process_param1_name.value)
            if process_param2_name.value:
                param_bounds.append((float(process_param2_min.value), float(process_param2_max.value)))
                param_names.append(process_param2_name.value)
            
            if not param_bounds:
                status_display.value = "<p style='color: red;'>Please specify parameter bounds</p>"
                return
            
            # Objective function
            def objective_func(params):
                try:
                    # For demonstration, compute metrics
                    # In practice, this would simulate process with params
                    metrics = compute_all_metrics(current_volume, voxel_size)
                    value = metrics.get(target_metric, 0.0)
                    return -value if maximize else value
                except:
                    return 1e10 if maximize else -1e10
            
            progress_bar.value = 30
            
            # Run optimization
            from scipy.optimize import differential_evolution, minimize as scipy_minimize
            
            if method == 'Differential Evolution':
                result = differential_evolution(objective_func, param_bounds, seed=42, maxiter=50)
            else:
                x0 = [(b[0] + b[1]) / 2.0 for b in param_bounds]
                if method == 'Nelder-Mead':
                    result = scipy_minimize(objective_func, x0, method='Nelder-Mead', bounds=param_bounds)
                else:  # BFGS
                    result = scipy_minimize(objective_func, x0, method='L-BFGS-B', bounds=param_bounds)
            
            progress_bar.value = 80
            
            optimal_params = dict(zip(param_names, result.x))
            optimal_value = -result.fun if maximize else result.fun
            
            virtual_experiment_results['optimization'] = {
                'optimal_params': optimal_params,
                'optimal_value': optimal_value,
                'method': method,
                'target_metric': target_metric
            }
            
            # Display results
            html = f"""
            <h4>🎯 Optimization Results</h4>
            <p><b>Objective:</b> {objective}</p>
            <p><b>Method:</b> {method}</p>
            <p><b>Optimal {target_metric}:</b> {optimal_value:.4f}</p>
            <h5>Optimal Parameters:</h5>
            <ul>
            """
            for param_name, param_value in optimal_params.items():
                html += f"<li><b>{param_name}:</b> {param_value:.4f}</li>"
            html += "</ul>"
            optimization_results_display.value = html
            
            # Visualize
            with optimization_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 1, figsize=(10, 6))
                
                # Create parameter space visualization
                if len(param_names) == 2:
                    # 2D contour plot
                    x_range = np.linspace(param_bounds[0][0], param_bounds[0][1], 20)
                    y_range = np.linspace(param_bounds[1][0], param_bounds[1][1], 20)
                    X, Y = np.meshgrid(x_range, y_range)
                    Z = np.zeros_like(X)
                    
                    for i in range(X.shape[0]):
                        for j in range(X.shape[1]):
                            Z[i, j] = -objective_func([X[i, j], Y[i, j]]) if maximize else objective_func([X[i, j], Y[i, j]])
                    
                    contour = axes.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)
                    axes.scatter([optimal_params[param_names[0]]], [optimal_params[param_names[1]]], 
                               s=300, color='red', marker='*', zorder=5, label='Optimal')
                    axes.set_xlabel(param_names[0], fontsize=11)
                    axes.set_ylabel(param_names[1], fontsize=11)
                    axes.set_title(f'Optimization: {target_metric}', fontsize=12, fontweight='bold')
                    axes.legend()
                    plt.colorbar(contour, ax=axes)
                    axes.grid(True, alpha=0.3)
                else:
                    # Bar chart for single parameter
                    axes.bar(param_names, [optimal_params[p] for p in param_names], alpha=0.7)
                    axes.set_ylabel('Parameter Value', fontsize=11)
                    axes.set_title(f'Optimal Parameters: {target_metric} = {optimal_value:.4f}', 
                                 fontsize=12, fontweight='bold')
                    axes.grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Optimization complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in optimization: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    load_button.on_click(load_volume_callback)
    run_sensitivity_button.on_click(run_sensitivity_callback)
    generate_doe_button.on_click(generate_doe_callback)
    run_optimization_button.on_click(run_optimization_callback)
    
    # Enable/disable widgets based on method selection
    def update_sensitivity_widgets(change):
        method = sensitivity_method.value
        n_trajectories.disabled = (method != 'Morris Screening')
        n_samples_sobol.disabled = (method != 'Sobol Indices')
    
    def update_doe_widgets(change):
        method = doe_method.value
        n_samples_lhs.disabled = (method != 'Latin Hypercube')
        process_param1_levels.disabled = (method != 'Full Factorial')
        process_param2_levels.disabled = (method != 'Full Factorial')
        process_param3_levels.disabled = (method != 'Full Factorial')
    
    sensitivity_method.observe(update_sensitivity_widgets, names='value')
    doe_method.observe(update_doe_widgets, names='value')
    
    print("✅ All callback functions attached!")


✅ All callback functions attached!


## 5. Display Interactive Dashboard


In [6]:
if WIDGETS_AVAILABLE:
    
    # Create loading panel
    loading_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load Segmented Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        HBox([load_button, volume_info_display])
    ])
    
    # Create sensitivity analysis panel
    sensitivity_panel = widgets.VBox([
        widgets.HTML("<h3>🔬 Sensitivity Analysis</h3>"),
        sensitivity_method,
        metric_to_analyze,
        widgets.HTML("<b>Parameter 1:</b>"),
        HBox([param1_name, param1_min, param1_max, param1_steps]),
        widgets.HTML("<b>Parameter 2 (optional):</b>"),
        HBox([param2_name, param2_min, param2_max, param2_steps]),
        HBox([n_trajectories, n_samples_sobol]),
        run_sensitivity_button,
        sensitivity_results_display,
        sensitivity_visualization
    ])
    
    # Create DoE panel
    doe_panel = widgets.VBox([
        widgets.HTML("<h3>📊 Design of Experiments</h3>"),
        doe_method,
        widgets.HTML("<b>Process Parameters:</b>"),
        HBox([process_param1_name, process_param1_min, process_param1_max, process_param1_levels]),
        HBox([process_param2_name, process_param2_min, process_param2_max, process_param2_levels]),
        HBox([process_param3_name, process_param3_min, process_param3_max, process_param3_levels]),
        n_samples_lhs,
        generate_doe_button,
        doe_results_display,
        doe_visualization
    ])
    
    # Create optimization panel
    optimization_panel = widgets.VBox([
        widgets.HTML("<h3>🎯 Parameter Optimization</h3>"),
        optimization_objective,
        optimization_method,
        run_optimization_button,
        optimization_results_display,
        optimization_visualization
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        sensitivity_panel,
        doe_panel,
        optimization_panel
    ])
    analysis_tabs.set_title(0, '🔬 Sensitivity')
    analysis_tabs.set_title(1, '📊 DoE')
    analysis_tabs.set_title(2, '🎯 Optimization')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>🔬 Advanced Analysis - Sensitivity and Virtual Experiments</h1>"),
        loading_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📊 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start analyzing sensitivity and designing virtual experiments.")
    print("\n💡 Tips:")
    print("   1. Load a segmented volume")
    print("   2. Run sensitivity analysis to identify key parameters")
    print("   3. Generate DoE designs for efficient experimentation")
    print("   4. Optimize process parameters for target metrics")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start analyzing sensitivity and designing virtual experiments.

💡 Tips:
   1. Load a segmented volume
   2. Run sensitivity analysis to identify key parameters
   3. Generate DoE designs for efficient experimentation
   4. Optimize process parameters for target metrics


## 6. Summary

### What We Learned

1. **Sensitivity Analysis**:
   - Parameter sweep to understand parameter effects
   - Local sensitivity (derivatives) for linear regions
   - Morris screening for global sensitivity ranking
   - Sobol indices for variance decomposition

2. **Design of Experiments (DoE)**:
   - Full factorial designs for complete exploration
   - Latin Hypercube Sampling for efficient space-filling
   - Central Composite Design for response surface modeling
   - Box-Behnken Design for quadratic models

3. **Parameter Optimization**:
   - Single-objective optimization
   - Multiple optimization algorithms (DE, Nelder-Mead, BFGS)
   - Response surface visualization
   - Optimal parameter identification

### Key Insights

- **Sensitivity analysis** helps identify which parameters matter most
- **Morris screening** is efficient for ranking many parameters
- **DoE methods** reduce the number of experiments needed
- **Optimization** finds best parameters for target metrics
- **Process-structure relationships** can be modeled and optimized

### Next Steps

- **Notebook 06**: Comparative Analysis and Batch Processing
  - Compare multiple samples
  - Batch processing workflows
  - Statistical comparison

- **Notebook 07**: Quality Control and Validation
  - Dimensional accuracy
  - Uncertainty quantification
  - Validation against ground truth

### Resources

- [Framework Documentation](../docs/README.md)
- [Analysis Modules](../docs/modules.md#analysis-modules)
- [Sensitivity Analysis](../docs/modules.md#analysissensitivity_analysis)
- [Virtual Experiments](../docs/modules.md#analysisvirtual_experiments)
